# Import libraries

In [ ]:
#libraries import
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
try:
  from unidecode import unidecode
except:
  !pip install unidecode
  from unidecode import unidecode
from scipy.stats import norm
from datetime import datetime

from openpyxl.utils.cell import column_index_from_string, get_column_letter
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import NamedStyle
import openpyxl

import shutil

from google.colab import files

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.1 MB/s eta 0:00:00


In [ ]:
# Fazer upload de um arquivo
uploaded = files.upload()

!mkdir -p Mews_Partners

# Mover o arquivo para um diretório específico
shutil.move(list(uploaded.keys())[0], '/content/Mews_Partners/')

Saving DataModeleComplet (5).xlsx to DataModeleComplet (5).xlsx


'/content/Mews_Partners/DataModeleComplet (5).xlsx'

## Functions

In [ ]:
def fix_column_names(df):
    dict_fix = {'.':' ','-':' ','/':' ',' ':'_'}
    column_names = df.columns
    new_column_names = []
    for col in column_names:
        # Remove accents and special characters
        fixed_col = unidecode(col)
        for char in dict_fix:
            fixed_col = fixed_col.replace(char, dict_fix[char])
        new_column_names.append(fixed_col)
    return new_column_names

def classeABCQuantity(df, qty_col_name, price_col_name):
    # sum of quantities sold in each year
    df = df[df[qty_col_name]!=0].reset_index(drop=True)
    df_total_qty = df.groupby(['Code', 'Year'])[qty_col_name].sum().reset_index()
    df_total_price = df.groupby(['Code', 'Year'])[price_col_name].sum().reset_index()

    # cumulative sum for each year
    df_total_qty = df_total_qty.sort_values(['Year',qty_col_name], ascending=[True, False])
    df_total_qty['Cumulative_Sum'] = df_total_qty.groupby('Year')[qty_col_name].cumsum()
    df_total_qty['Cumulative_Percentage'] = df_total_qty['Cumulative_Sum'] / df_total_qty.groupby('Year')[qty_col_name].transform('sum')

    # classify the product in classes A,B and C
    df = pd.merge(df, df_total_qty[['Code', 'Year', 'Cumulative_Percentage']], on=['Code', 'Year'])
    abc_categories = pd.cut(df['Cumulative_Percentage'], bins=[0, 0.8, 0.95, 1.0], labels=['A', 'B', 'C'])
    df['Classe_ABC_quantity'] = abc_categories

    # cumulative sum for each year
    df_total_price = df_total_price.sort_values(['Year',price_col_name], ascending=[True, False])
    df_total_price['Cumulative_Sum_revenue'] = df_total_price.groupby('Year')[price_col_name].cumsum()
    df_total_price['Cumulative_Percentage_revenue'] = df_total_price['Cumulative_Sum_revenue'] / df_total_price.groupby('Year')[price_col_name].transform('sum')

    # classify the product in classes A,B and C
    df = pd.merge(df, df_total_price[['Code', 'Year', 'Cumulative_Percentage_revenue']], on=['Code', 'Year'])
    abc_categories = pd.cut(df['Cumulative_Percentage_revenue'], bins=[0, 0.8, 0.95, 1.0], labels=['A', 'B', 'C'])
    df['Classe_ABC_revenue'] = abc_categories

    return df

def fix_dates(df):
  df['Date'] = df['Year'].astype(str) + '-' + df['Month'].astype(str)
  df.Date = pd.to_datetime(df.Date, format='%Y-%m')
  return df

def calculate_delay(df, list_col_delay, list_col_freq):
  # calculating the delay time
  dict_time = {'jrs': 1/30, 'semaine': 7/30, 'semaines': 7/30, 'mois':1}
  for col_delay, col_freq in zip(list_col_delay, list_col_freq):
    df[col_delay] = df[col_delay].map(lambda x: x.split())
    df[col_delay] = df[col_delay].map(lambda x: int(x[0])*dict_time[x[1]])
    df[col_freq] = df[col_freq].map(lambda x: x.split())
    df[col_freq] = df[col_freq].map(lambda x: int(x[0])*dict_time[x[1]])
  return df

def calculate_delay(df, list_col):
  # calculating the delay time
  dict_time = {'jrs': 1/30, 'semaine': 7/30, 'semaines': 7/30, 'mois':1}
  for col in list_col:
    df[col] = df[col].map(lambda x: x.split())
    df[col] = df[col].map(lambda x: int(x[0])*dict_time[x[1]])
  return df

# Import data

In [ ]:
df_base_prev_passe_original = gc.open("DataModeleComplet").worksheet("Previsions_historique")
df_base_historique_original = gc.open("DataModeleComplet").worksheet("Ventes_historique")
df_flux_original = gc.open("DataModeleComplet").worksheet("Baseflux")
df_base_article_original = gc.open("DataModeleComplet").worksheet("Basearticle")
df_stock_original = gc.open("DataModeleComplet").worksheet("Stocks")

# get_all_values gives a list of rows.
df_base_prev_passe_original = df_base_prev_passe_original.get_all_values()
df_base_historique_original = df_base_historique_original.get_all_values()
df_flux_original = df_flux_original.get_all_values()
df_base_article_original = df_base_article_original.get_all_values()
df_stock_original = df_stock_original.get_all_values()

# Convert to a DataFrame and render.
pd.DataFrame.from_records(df_base_prev_passe_original)
pd.DataFrame.from_records(df_base_historique_original)
pd.DataFrame.from_records(df_flux_original)
pd.DataFrame.from_records(df_base_article_original)
pd.DataFrame.from_records(df_stock_original)

,0,1,2,3,4,5,6
0,,,,,,,
1,,,,,,,
2,,,,,,,
3,,,Article,Entrepôt,Année,Mois,Stock
4,,,1000010,Central,2018,2,412
...,...,...,...,...,...,...,...
49383,,,9900334,UK,2018,8,0
49384,,,9900334,UK,2018,9,0
49385,,,9900334,UK,2018,10,0
49386,,,9900334,UK,2018,11,0


In [ ]:
# importing data
df_base_prev_passe_original = pd.read_excel('Mews_Partners/DataModeleComplet.xlsx', header=2, sheet_name = 'Previsions_historique').iloc[1:, 3:]
df_base_historique_original = pd.read_excel('Mews_Partners/DataModeleComplet.xlsx', header=3, sheet_name = 'Ventes_historique').iloc[:, 2:]
df_flux_original = pd.read_excel('Mews_Partners/DataModeleComplet.xlsx', header=2, sheet_name = 'Baseflux').iloc[1:, 2:]
df_base_article_original = pd.read_excel('Mews_Partners/DataModeleComplet.xlsx', header=1, sheet_name = 'Basearticle')
df_stock_original = pd.read_excel('Mews_Partners/DataModeleComplet.xlsx', header=3, sheet_name = 'Stocks').iloc[:, 2:]

# Analysis

In [ ]:
# fixing columns and data types
df_base_prev_passe = df_base_prev_passe_original.copy()
new_cols = ['Code', 'Marche', 'Canal', 'Code_Magasin', 'Year', 'Month', 'Forecast_Magasin']
df_base_prev_passe.columns = new_cols

df_base_prev = df_base_prev_passe.copy()
df_base_prev.Code = df_base_prev.Code.astype(int)
df_base_prev.Year = df_base_prev.Year.astype(int)
df_base_prev.Month = df_base_prev.Month.astype(int)
df_base_prev = df_base_prev.replace("nan","").replace("-","")

df_base_historique = df_base_historique_original.copy()
df_base_historique.rename(columns = {'Code article ': 'Code', 'Marché':'Marche', 'Code point de stock': 'Code_Magasin', 'Année': 'Year',
                                              'Mois ': 'Month', 'Quantité': 'Historique_vente_Magasin'}, inplace = True)

df_base_historique = df_base_historique.replace("nan","")
df_base_historique.Canal = df_base_historique.Canal.astype(str)
df_base_historique.Code = df_base_historique.Code.astype(int)
df_base_historique.Year = df_base_historique.Year.astype(int)
df_base_historique.Month = df_base_historique.Month.astype(int)


df_flux = df_flux_original.copy()
list_cols_names_flux = ['Code', 'Marche','Canal','Delay_Magasin','Frequency_Magasin','MOQ_Magasin','Code_Magasin','Delay_France','Frequency_France','MOQ_France','Code_France','Delay_Monde','Frequency_Monde','MOQ_Monde','Code_Monde', 'Delay_Usine','Frequency_Usine','MOQ_Usine','Code_Usine']
df_flux.columns = list_cols_names_flux
df_flux = df_flux.replace("-","")

df_base_article = df_base_article_original.iloc[:,1:].copy()
df_base_article.columns = fix_column_names(df_base_article)
df_base_article = df_base_article.rename(columns={'Code_article':'Code','Classe_':'Classe_base'})

df_stock = df_stock_original.copy()
df_stock.rename(columns = {'Article': 'Code', 'Site ': 'Entrepot', 'Année': 'Year', 'Mois': 'Month', 'Stock':'Stock_begin_month'}, inplace = True)

df_base_prev_ventes = df_base_prev[['Code','Marche','Year','Month','Forecast_Magasin']].merge(df_base_historique,on=['Code','Marche','Year','Month'])
df_base_prev_ventes = fix_dates(df_base_prev_ventes)
df_stock = fix_dates(df_stock).drop(['Year', 'Month'], axis=1)

# adding a month on Stock date, to facilitate further analysis
df_stock.Date = df_stock.Date + pd.DateOffset(months=1)

df_prev_ventes_stock = df_base_prev_ventes.merge(df_stock, on=['Code', 'Date'])

# fixing the names of the entrepôts
list_entrepots = {'France':'FR','Belgique':'BE','Belgium':'BE','Spain':'ES','Turkey':'TK','Germany':'DE', 'Portugal':'PT', 'Italy':'IT', 'UK':'UK', 'Swiss':'CH', 'Poland':'PL'}
df_prev_ventes_stock['Marche'] = df_prev_ventes_stock['Marche'].replace(list_entrepots)
df_flux['Marche'] = df_flux['Marche'].replace(list_entrepots)

df_flux = df_flux[df_flux.Code != "Loading..."]
df_prev_ventes_stock = df_prev_ventes_stock[df_prev_ventes_stock.Code != "Loading..."]

In [ ]:
df_prev_ventes_stock[(df_prev_ventes_stock.Code==1000055)].

## Fill not available data in forecast with its mean

In [ ]:
# if historique vente < 0 and forecast is None, forecast is 0
df_prev_ventes_stock_clean = df_prev_ventes_stock.copy()
df_prev_ventes_stock_clean.loc[(df_prev_ventes_stock_clean['Historique_vente_Magasin']<=0)&(df_prev_ventes_stock_clean['Forecast_Magasin'].isna()),
                               'Forecast_Magasin'] = 0

# making that every negative number on Historique vente is 0
df_prev_ventes_stock_clean.loc[df_prev_ventes_stock_clean.query('Historique_vente_Magasin<0').index,'Historique_vente_Magasin'] = 0

# calculating the mean value of forecast respecting the product and the entrepot
mean_forecast = df_prev_ventes_stock_clean[(df_prev_ventes_stock_clean['Forecast_Magasin']>0)].groupby(['Code','Entrepot','Year'])[[
    'Forecast_Magasin']].mean().reset_index()

df_forecast_pos = df_prev_ventes_stock_clean[df_prev_ventes_stock_clean['Forecast_Magasin']>0].copy()
df_forecast_nan = df_prev_ventes_stock_clean[df_prev_ventes_stock_clean['Forecast_Magasin'].isna()].copy()
df_forecast_nan_mean = df_forecast_nan.drop('Forecast_Magasin', axis=1).merge(mean_forecast, on=['Code','Entrepot','Year'])
df_merge_forecast = pd.concat([df_forecast_pos,df_forecast_nan_mean])

# Joining data in entrepôt central and local

In [ ]:
df_merge_forecast.Code_Magasin = df_merge_forecast.Code_Magasin.fillna("")

df_forecast_local_central = df_merge_forecast.groupby(['Code','Year','Month','Entrepot','Canal','Code_Magasin','Date','Stock_begin_month']).aggregate({'Forecast_Magasin':'sum','Historique_vente_Magasin':'sum'}).reset_index()
df_forecast_local_central.drop_duplicates(inplace=True)

## Calculate the Standard deviation

In [ ]:
df_merge_forecast_clean = df_forecast_local_central.copy()
df_forecast_article = df_merge_forecast_clean.merge(df_base_article[['Code','Prix_de_vente','PRI','Classe_base']], on='Code')
df_forecast_article['Revenue_historic'] = df_forecast_article['Historique_vente_Magasin']*df_forecast_article['Prix_de_vente']
df_forecast_article['Revenue_forecasted'] = df_forecast_article['Forecast_Magasin']*df_forecast_article['Prix_de_vente']

# selectionating only data with Historique vente and Forecast
df_diff = df_forecast_article.loc[(df_forecast_article.Historique_vente_Magasin>=0)&(df_forecast_article.Forecast_Magasin>=0)][[
    'Code','Historique_vente_Magasin','Entrepot','Forecast_Magasin','Revenue_historic','Revenue_forecasted','Year','Month']].copy()

df_diff['Std_deviation_qty'] = df_diff['Historique_vente_Magasin'] - df_diff['Forecast_Magasin']

df_diff['Std_deviation_rev'] = df_diff['Revenue_forecasted'] - df_diff['Revenue_historic']

df_diff_std = df_diff.groupby(['Code','Entrepot','Year'])[['Std_deviation_qty','Std_deviation_rev']].std().reset_index()

# merge dataframes
df_base_std = pd.merge(df_forecast_article, df_diff_std, on=['Entrepot','Code','Year'])
df_base_std = df_base_std[(~df_base_std.Std_deviation_qty.isna())&(~df_base_std.Std_deviation_rev.isna())]

## Calculate the Stock Roulant

In [ ]:
df_marche = df_merge_forecast[['Code','Marche','Year','Entrepot']].drop_duplicates()

df_base_std = df_base_std.merge(df_marche, on=['Code','Year','Entrepot'])

In [ ]:
df_std_flux = df_base_std[['Code', 'Marche', 'Year', 'Month', 'Forecast_Magasin','Historique_vente_Magasin','Revenue_historic','Revenue_forecasted',
                           'Date', 'Entrepot','Stock_begin_month', 'Std_deviation_qty','Std_deviation_rev','Prix_de_vente', 'PRI', 'Classe_base']].merge(
                               df_flux, on=['Code','Marche'])

list_cols = ['Delay_Magasin', 'Delay_France', 'Delay_Monde', 'Delay_Usine','Frequency_Magasin', 'Frequency_France', 'Frequency_Monde', 'Frequency_Usine']
df_std_flux[list_cols] = df_std_flux[list_cols].replace("","0 jrs").fillna("0 jrs")

df_std_delay = calculate_delay(df_std_flux, list_cols)

# calculate stock roulant
df_mean_forecast = df_std_delay[df_std_delay.Forecast_Magasin>0].groupby(['Code','Year','Entrepot'])[[
    'Forecast_Magasin']].mean().reset_index().rename(columns={'Forecast_Magasin':'Mean_forecast'})

df_stock_roulant = df_std_delay.merge(df_mean_forecast, on=['Code','Year','Entrepot'])

df_stock_roulant['Frequency'] = df_stock_roulant['Frequency_Magasin'] + df_stock_roulant['Frequency_France'] + df_stock_roulant['Frequency_Monde'] + df_stock_roulant['Frequency_Usine']
df_stock_roulant['Delay'] = df_stock_roulant['Delay_Magasin'] + df_stock_roulant['Delay_France'] + df_stock_roulant['Delay_Monde'] + df_stock_roulant['Delay_Usine']

# the stock roulant is calculated ny multiplying the mean forecast in a year x frequency / 2
df_stock_roulant['Stock_roulant'] = df_stock_roulant['Frequency']*df_stock_roulant['Mean_forecast']/2

## Calculate the ABC class for each product

In [ ]:
# classify the products in quantity and in revenue
df_classes = classeABCQuantity(df_stock_roulant, 'Historique_vente_Magasin', 'Revenue_historic')[
    ['Code', 'Year','Cumulative_Percentage', 'Classe_ABC_quantity', 'Cumulative_Percentage_revenue',
       'Classe_ABC_revenue']].drop_duplicates(subset=['Code','Year'])

df_merge_forecast_std = df_stock_roulant.merge(df_classes, on=['Code','Year'])

## Update Google Sheets file

In [ ]:
# df to save in google sheets
df_save = df_merge_forecast_std[['Date', 'Entrepot', 'Code',
       'Historique_vente_Magasin', 'Forecast_Magasin', 'Stock_begin_month',
       'Std_deviation_rev',
       'Classe_ABC_quantity',
       'Classe_ABC_revenue', 'Canal', 'Delay',
       'Frequency', 'Stock_roulant','Std_deviation_qty','Delay_Magasin', 'Frequency_Magasin', 'MOQ_Magasin',
       'Delay_France', 'Frequency_France', 'MOQ_France',
       'Delay_Monde', 'Frequency_Monde', 'MOQ_Monde',
       'Delay_Usine', 'Frequency_Usine', 'MOQ_Usine']]

df_save.drop_duplicates(subset=['Date', 'Entrepot', 'Code'],inplace=True)

from google.colab import drive
drive.mount('/content/drive')

from google.colab import auth
from google.auth import default
import gspread
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
spreadsheet = gc.open('Pole_Mews')
worksheet = spreadsheet.worksheet('Stock Roulant')

df_save['Date'] = df_save['Date'].astype(str)
df_save = df_save.fillna('')
worksheet.update('A1', [df_save.columns.values.tolist()] + df_save.values.tolist())

<ipython-input-112-cb60f41e8b13>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_save.drop_duplicates(subset=['Date', 'Entrepot', 'Code'],inplace=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-112-cb60f41e8b13>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_save['Date'] = df_save['Date'].astype(str)


{'spreadsheetId': '1om5mvcsoMMgb4DooKN_gHZOXXp0miojrTm7ZL1OqPxc',
 'updatedRange': "'Stock Roulant'!A1:Z12635",
 'updatedRows': 12635,
 'updatedColumns': 26,
 'updatedCells': 328510}